# Copyright Netherlands eScience Center <br>
** Function     : Packing netCDF for the vertical profile of fields (pressure level) from ERA-Interim** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2018.10.29 ** <br>
** Last Update  : 2018.10.29** <br>
Description     : This notebook aims to pack the vertical profile of 3D fieldsfrom ERA-Interim into netCDF files.<br>
Return Values   : netCDF4 <br>
Caveat          : <br>

In [ ]:
import numpy as np
import scipy as sp
import time as tttt
from netCDF4 import Dataset,num2date
import os

In [ ]:
# constants
constant = {'g' : 9.80616,      # gravititional acceleration [m / s2]
            'R' : 6371009,      # radius of the earth [m]
            'cp': 1004.64,      # heat capacity of air [J/(Kg*K)]
            'Lv': 2264670,      # Latent heat of vaporization [J/Kg]
            'R_dry' : 286.9,    # gas constant of dry air [J/(kg*K)]
            'R_vap' : 461.5,    # gas constant for water vapour [J/(kg*K)]
            }

In [ ]:
################################   Input zone  ######################################
# specify starting and ending time
start_year = 1979
end_year = 2017
# specify data path
# ERAI 3D fields on pressure level
datapath_3D = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ERAI/regression/pressure/monthly'
# specify output path for figures
output_path = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ERAI/regression'
####################################################################################

In [ ]:
def var_key_retrieve(datapath, year):
    # get the path to each datasets
    print ("Start retrieving datasets %d (y)" % (year))
    # The shape of each variable is (121,480)
    datapath_full = os.path.join(datapath, 'pressure_monthly_075_%d_u_T_z.nc' % (year))
    # get the variable keys
    var_key = Dataset(datapath_full)

    print ("Retrieving datasets successfully and return the variable key!")
    return var_key

In [ ]:
def create_netcdf_point (pool_t_vert, pool_z_vert, pool_du_vert, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    #logging.info("Start creating netcdf file for the 2D fields of ERAI at each grid point.")
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(os.path.join(output_path, 'pressure_erai_monthly_regress_1979_2017_vertProfile_du_t_z.nc'),'w',format = 'NETCDF4')
    # create dimensions for netcdf data
    year_wrap_dim = data_wrap.createDimension('year',Dim_year)
    month_wrap_dim = data_wrap.createDimension('month',Dim_month)
    lat_wrap_dim = data_wrap.createDimension('latitude',Dim_latitude)
    lev_wrap_dim = data_wrap.createDimension('level',Dim_level)
    # create coordinate variable
    year_wrap_var = data_wrap.createVariable('year',np.int32,('year',))
    month_wrap_var = data_wrap.createVariable('month',np.int32,('month',))
    lat_wrap_var = data_wrap.createVariable('latitude',np.float32,('latitude',))
    lev_wrap_var = data_wrap.createVariable('level',np.int32,('level',))
    # create the actual 4d variable
    t_vert_wrap_var = data_wrap.createVariable('t_vert',np.float64,('year', 'month', 'level', 'latitude'),zlib=True)
    z_vert_wrap_var = data_wrap.createVariable('z_vert',np.float64,('year', 'month', 'level', 'latitude'),zlib=True)
    du_vert_wrap_var = data_wrap.createVariable('du_vert',np.float64,('year', 'month', 'level', 'latitude'),zlib=True)
    # global attributes
    data_wrap.description = 'Monthly mean vertical profile of fields from ERA-Interim on pressure level'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lev_wrap_var.units = 'hPa'

    t_vert_wrap_var.units = 'K'
    z_vert_wrap_var.units = 'm2/s2'
    du_vert_wrap_var.units = '/s'

    t_vert_wrap_var.long_name = 'temperature'
    z_vert_wrap_var.long_name = 'geopotential'
    du_vert_wrap_var.long_name = 'zonal wind vertical shear'

    # writing data
    lat_wrap_var[:] = latitude
    lev_wrap_var[:] = level
    month_wrap_var[:] = index_month
    year_wrap_var[:] = period

    t_vert_wrap_var[:] = pool_t_vert
    z_vert_wrap_var[:] = pool_z_vert
    du_vert_wrap_var[:] = pool_du_vert

    # close the file
    data_wrap.close()
    print ("The generation of netcdf files for fields on surface is complete!!")

In [ ]:
def retriver(key):
    print ('Extract monthly mean fields.')
    u = var_key.variables['u'][:]
    t = var_key.variables['t'][:]
    gz = var_key.variables['z'][:]
    lev = var_key.variables['level'][:]
    # calculate the height
    z = gz / constant['g']
    # create arrays to store the values
    du = np.zeros(u.shape,dtype=float)  
    for i in np.arange(len(level)-2):
        du[:,i+1,:,:] = (u[:,i,:,:] - u[:,i+2,:,:]) / (z[:,i,:,:] - z[:,i+2,:,:])
    # take the vertical profile
    t_vert = np.mean(t,3)
    gz_vert = np.mean(gz,3)
    du_vert = np.mean(du,3)

    return du_vert, t_vert, gz_vert

In [ ]:
if __name__=="__main__":
    ####################################################################
    ######  Create time namelist matrix for variable extraction  #######
    ####################################################################
    # date and time arrangement
    # namelist of month and days for file manipulation
    namelist_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
    # index of months
    period = np.arange(start_year,end_year+1,1)
    index_month = np.arange(1,13,1)
    ####################################################################
    ######       Extract invariant and calculate constants       #######
    ####################################################################
    # get invariant from benchmark file
    Dim_year = len(period)
    Dim_month = len(index_month)
    Dim_latitude = 121
    Dim_longitude = 480
    Dim_level = 37
    #############################################
    #####   Create space for stroing data   #####
    #############################################
    # data pool
    pool_du = np.zeros((Dim_year, Dim_month, Dim_level, Dim_latitude),dtype = float)
    pool_t = np.zeros((Dim_year, Dim_month, Dim_level, Dim_latitude),dtype = float)
    pool_z = np.zeros((Dim_year, Dim_month, Dim_level, Dim_latitude),dtype = float)
    latitude = np.zeros(Dim_latitude,dtype=float)
    level = np.zeros(Dim_level,dtype=int)
    # loop for calculation
    for i in period:
        # get the key of each variable
        var_key = var_key_retrieve(datapath_3D,i)
        latitude = var_key.variables['latitude'][:]
        level = var_key.variables['level'][:]
        du, t, z = retriver(var_key)
        pool_du[i-1979,:,:,:] = du
        pool_t[i-1979,:,:,:] = t
        pool_z[i-1979,:,:,:] = z
    ####################################################################
    ######                 Data Wrapping (NetCDF)                #######
    ####################################################################
    create_netcdf_point(pool_t, pool_z, pool_du, output_path)
    print ('Packing 3D fields of ERA-Interim on pressure level is complete!!!')
    print ('The output is in sleep, safe and sound!!!')